In [1]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
import catboost as catboost

from catboost import CatBoostRegressor

driver = pd.read_csv("/content/driver.csv")

df_historic = pd.read_csv("/content/data_graph.csv")

modelDist = CatBoostRegressor()
modelDist.load_model("catboost_modelDist.cbm")

modelEarn = CatBoostRegressor()
modelEarn.load_model("catboost_modelEarn.cbm")

modelLocLocCity = CatBoostRegressor()
modelLocLocCity.load_model("catboost_modelLocLocCity.cbm")

modelTime = CatBoostRegressor()
modelTime.load_model("catboost_modelTime.cbm")

In [3]:
driver = driver.head(1)

In [4]:
driver["product"]=driver["productType"]
driver["vehicle_type"]=driver["vehicleType"]
driver["is_ev"]=driver["isEv"]
driver["earner_type"]=driver["earnerType"]
driver["fuel_type"]=driver["fuelType"]
driver["weather"]=driver["weatherType"]
driver["experience_months"]=driver["experienceMonths"]

In [5]:
df_historic.head()

,earner_type,city_id,product,start_time,pickup_lat,pickup_lon,drop_lat,drop_lon,surge_multiplier,payment_type
0,0,3,0,23,52.09319,5.17695,52.08017,5.16125,1.22,0
1,0,4,0,5,51.41972,5.50334,51.46578,5.52210,1.01,1
2,0,3,1,18,52.05469,5.11900,52.10705,5.13061,1.36,1
3,0,2,2,23,51.94535,4.51494,51.90919,4.45792,1.22,2
4,0,3,0,18,52.10560,5.13254,52.05785,5.06818,1.36,2


In [6]:
df_historic = df_historic[df_historic["earner_type"] == driver["earnerType"].iloc[0]]

In [7]:
driver = driver.head(1)

In [8]:
df_historic["vehicle_type"] = driver["vehicle_type"]
df_historic["is_ev"] = driver["is_ev"]
df_historic["fuel_type"] = driver["fuel_type"]
df_historic["experience_months"] = driver["experience_months"]
df_historic["rating"] = driver["rating"]
df_historic["weather"] = driver["weather"]

In [9]:
df_historic = df_historic[['city_id', 'product', 'vehicle_type', 'is_ev', 'start_time',
       'pickup_lat', 'pickup_lon', 'drop_lat', 'drop_lon', 'surge_multiplier',
       'payment_type', 'weather', 'earner_type', 'fuel_type',
       'experience_months', 'rating']]

In [10]:
df_aux = pd.DataFrame()

df_aux["timeBetween"] = modelTime.predict(df_historic)

df_save = df_historic[['pickup_lat', 'pickup_lon', 'drop_lat', 'drop_lon']]

df_historic["pickup_lat"] = driver["driverLat"]
df_historic["pickup_lon"] = driver["driverLon"]
df_historic["drop_lat"] = df_save["pickup_lat"]
df_historic["drop_lon"] = df_save["pickup_lon"]

df_aux["timeTo"] = modelTime.predict(df_historic)

df_historic["pickup_lat"] = df_save["drop_lat"]
df_historic["pickup_lon"] = df_save["drop_lon"]
df_historic["drop_lat"] = driver["driverLat"]
df_historic["drop_lon"] = driver["driverLon"]

df_aux["timeBack"] = modelTime.predict(df_historic)
df_aux["earnGet"] = modelEarn.predict(df_historic)

In [11]:
df_aux["time"] = df_historic["start_time"]

df_aux["calc"] = df_aux["earnGet"]/((df_aux["timeTo"]+df_aux["timeBetween"]+df_aux["timeBack"])/60)

In [12]:
df_aux.head()

,timeBetween,timeTo,timeBack,earnGet,time,calc
0,19.821566,16.514727,14.238832,10.794959,23,12.806642
1,12.889155,29.146022,24.002141,6.874863,5,6.246343
2,28.868235,21.422923,10.409129,11.516515,18,11.383651
3,25.129461,29.155602,23.145346,9.278476,23,7.189792
4,20.948823,32.645730,12.237701,9.648225,18,8.793463


In [13]:
avgTime = []

for i in range(0, 24):
  df_time = df_aux[df_aux["time"]==i]

  df_time = df_time.sort_values(by='calc', ascending=False)

  #print(df_time["calc"].iloc[0:11])

  avgTime.append(sum(df_time["calc"].iloc[0:11])/10)

In [14]:
auxTime = avgTime.copy()

In [15]:
auxTime.sort()

In [16]:
treshold = auxTime[8]

In [17]:
ans = np.array(avgTime)

ans = ans >= treshold

In [18]:
print(ans, end="")

[ True False False False False False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False False  True False]